In [82]:
import re
import os
import networkx as nx
from pyvis.network import Network

In [83]:
BASE_PATH = os.path.abspath("..")

In [84]:
PACKAGE_FILTER = "^(instaloader|Instaloader).*"
CG_JSON = f"{BASE_PATH}/cg_jsons/instaloader_cg.json"
OUTPUT_FILE_PATH = f"{BASE_PATH}/cg_htmls/instaloader_graph.html"

In [85]:
import json
with open(CG_JSON, mode="r") as f:
    data = json.load(f)

In [86]:
def clean_json(dictionary : dict, package_filter: str) -> dict:
    '''
        Change the format of all file paths to standard package naming convention in Python
        and build network graph
    '''
    new_dict = {}
    for key, value in dictionary.items():
       
       # TO-DO: You can replace any characters before src with blank
       new_key = re.sub(r"[\\]+", ".", key.strip(".\\"))  # Strip chars and replace slashes
       new_values = [re.sub(r"[\\]+", ".", elm.strip(".\\")) for elm in value] # Strip chars and replace slashes
       
    #    # Only keep webdriver related packages in "from" node
    #    if not re.match(package_filter, new_key):
    #        continue
       
    #    # Only keep webdriver related packages in "to" node
    #    new_values = list(filter(lambda x: re.match(package_filter, x), new_values))
       
       if new_key in new_dict:
           new_dict[new_key].extend(new_values)
       else:
           new_dict[new_key] = new_values

    return new_dict        


In [87]:
new_json = clean_json(data, PACKAGE_FILTER)

In [88]:
[key for key in new_json.keys() if re.match("^instaloader.__main__._main", key)]

['instaloader.__main__._main']

In [89]:
def toNetwork(data: dict)->  nx.DiGraph:
    nt = nx.DiGraph()

    def checkKey(name):
        if name not in nt:
            nt.add_node(name, size=40)

    for node in data:
        checkKey(node)
        for child in data[node]:
            checkKey(child)
            nt.add_edge(node,child)
    return nt

def ntw_pyvis(ntx:nx.DiGraph):
    net = Network(width="100%",height="1000px", directed=True, notebook=True,
                  cdn_resources='remote', select_menu=True, filter_menu=True)
    for node in ntx.nodes:
        net.add_node(node, **{"label":node},)

    for edge in ntx.edges:
        net.add_edge(edge[0], edge[1], width=1)
    return net

In [90]:
network = toNetwork(new_json)

In [91]:
print("Number of Nodes:", len(network.nodes))
print("Number of Edges:", len(network.edges))

Number of Nodes: 805
Number of Edges: 1136


In [92]:
net = ntw_pyvis(network)

In [93]:
# net.show_buttons(filter_=['physics'])

In [94]:
options = """
const options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -6300
    },
    "minVelocity": 0.75
  }
}
"""
net.set_options(options)

In [95]:
_ = net.show(name=OUTPUT_FILE_PATH)

c:\Users\harri\Documents\Programming\call_graph/cg_htmls/instaloader_graph.html
